## Submission Part#1 : Web Scraping and Getting Toronto Neighbourhood Data into Pandas Dataframe

# Install or Check Requested libraries required for Web Scraping 

In [1]:
!pip install beautifulsoup4

In [2]:
!pip install lxml

In [3]:
!pip install requests

# Import Libraries

In [4]:
from bs4 import BeautifulSoup # case sensitive name!
import requests
import lxml
import pandas as pd
import numpy as np

# Sour HTML data or files into Text and Parse it using BeautifulSoup 

In [5]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml') # parser lxml 

In [6]:
# get the table rows
trow = soup.find('table', class_='wikitable sortable')

In [7]:
trow.tr.text

'\nPostcode\nBorough\nNeighbourhood\n'

In [8]:
trow_cols = trow.tr.text.split("\n") # list
trow_cols.remove('') # remove blank element from list
trow_cols.remove('') # remove blank element from list
trow_cols

['Postcode', 'Borough', 'Neighbourhood']

In [9]:
# fist clear the list
collect_list = list()

# get the table rows
for trow in soup.find_all('tr'):
    trow_cols = trow.text.split("\n")
    trow_cols.remove('')
    if len(trow_cols) == 4:
        trow_cols.remove('')
    #print(len(trow_cols))
    if len(trow_cols) == 3:
        collect_list.append(trow_cols)
print(collect_list[0:1])

[['Postcode', 'Borough', 'Neighbourhood']]


In [10]:
df=pd.DataFrame(collect_list,columns=['Postcode', 'Borough', 'Neighborhood'])
df.drop(0, inplace=True)
df.shape

(287, 3)

In [11]:
df.head()

,Postcode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


In [12]:
df = df[df.Borough != 'Not assigned']  # Not assigned Bororugh removed 
df['Neighborhood'] = np.where(df['Neighborhood'] == 'Not assigned', df['Borough'], df['Neighborhood']) # neighbourhood not assigned replaced with Borough..
df.head()

,Postcode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor


Flat Dataset is ready to use further for aggr neighbourhood

In [13]:
df.shape

(210, 3)

Aggr neighbourhood

In [14]:
df_agg = df.sort_values(by=['Postcode', 'Borough']) # sort before listagg
df_agg =df_agg.groupby(['Postcode', 'Borough'])['Neighborhood'].agg([('Neighborhood', ', '.join)]).reset_index() #Neighborhood grouping!!
df_agg.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [15]:
df_agg[df_agg.Postcode == 'M1R' ] # Testing for one PostCode

,Postcode,Borough,Neighborhood
11,M1R,Scarborough,"Maryvale, Wexford"


# as per instruction, last command of notebook

In [16]:
df_agg.shape # as per instruction, last command of notebook

(103, 3)

## Submission Part#2 : Geospatial attributes fetch per postcode

#Geospatial_Coordinates.csv file loaded in using "Insert to Code" Funcion.

In [17]:
# The code was removed by Watson Studio for sharing.

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
df_new = df_agg.join(df_geo.set_index('Postcode'), on='Postcode') # join with Df_Geo geospatial dataset.
df_new.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Making sure the shape remain same as before

In [19]:
df_new.shape 

(103, 5)

## Submission Part#3 : Borough = Toronto and analysis

Import required libraries to plot data on map , K means clustering

In [20]:
!pip install geopy

In [21]:
!pip install folium

     |████████████████████████████████| 92kB 11.8MB/s eta 0:00:01


In [22]:
import json 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Old Flat File to be used to get neighbourhood one per line.

In [23]:
df.head()

,Postcode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor


Pick up Neighbourhood where it contains Toronto word in it

In [24]:
toronto_data = df[df['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data = toronto_data.join(df_geo.set_index('Postcode'), on='Postcode') # join with Df_Geo geospatial dataset.
toronto_data.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M9A,Downtown Toronto,Queen's Park,43.667856,-79.532242
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418


In [25]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(len(toronto_data['Borough'].unique()),toronto_data.shape[0]))

The dataframe has 4 boroughs and 74 neighborhoods.


Get Toronoto City Coordinates

In [26]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [27]:
#print(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood'])
#print(toronto_data['Longitude'].head(1))
#print(toronto_data['Borough'].head(1))
#print(toronto_data['Neighbourhood'].head(1))

In [29]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, Borough, Neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(Neighborhood, Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Foursquare Credetials and Set up

In [90]:
CLIENT_ID = 'xxxxxxxx' # your Foursquare ID
CLIENT_SECRET = 'xxxxx' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: xxxxxxxx
CLIENT_SECRET:xxxxx


Research First neighbourhood of Toronto

In [32]:
toronto_data.loc[0, 'Neighborhood']

'Harbourfront'

Get the neighborhood's latitude and longitude values.

In [33]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Harbourfront are 43.6542599, -79.3606359.


#### Now, let's get the top 50 venues that are in Marble Hill within a radius of 500 meters.

In [34]:

LIMIT = 50 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url 

'https://api.foursquare.com/v2/venues/explore?&client_id=PQ1TIS30AV0NJ3CKCODYXOEL03AD4AVD5ECQ0MNPJYKAHM4Q&client_secret=5HHHSC2KQLSO2VLCF5Q34KWOHMLKXSCO3JY52F1JZYYR2NZE&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=50'

In [35]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ded11a378a484001b018ff1'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 49,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

### function that extracts the category of the venue

In [36]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Preparing for Venue lists surrounding Neighborhood and converted Json file return into dataframe

In [37]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Gym / Fitness Center,43.653191,-79.357947
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149


#### Total venue searched

In [38]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

49 venues were returned by Foursquare.


#### Function to find out venues nearby each neighborhood from dataframe.

In [39]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [41]:
# Run the function thru all dataframe values

toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )


Harbourfront
Queen's Park
Ryerson
Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
The Danforth West
Riverdale
Design Exchange
Toronto Dominion Centre
Brockton
Exhibition Place
Parkdale Village
The Beaches West
India Bazaar
Commerce Court
Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North
Forest Hill West
High Park
The Junction South
North Toronto West
The Annex
North Midtown
Yorkville
Parkdale
Roncesvalles
Davisville
Harbord
University of Toronto
Runnymede
Swansea
Moore Park
Summerhill East
Chinatown
Grange Park
Kensington Market
Deer Park
Forest Hill SE
Rathnelly
South Hill
Summerhill West
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
St. James Town
First Canadian Place
Underground city

In [42]:
print(toronto_venues.shape)
toronto_venues.head()

(2210, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [43]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,50,50,50,50,50,50
Bathurst Quay,17,17,17,17,17,17
Berczy Park,50,50,50,50,50,50
Brockton,22,22,22,22,22,22
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
CN Tower,17,17,17,17,17,17
Cabbagetown,45,45,45,45,45,45
Central Bay Street,50,50,50,50,50,50
Chinatown,50,50,50,50,50,50


In [44]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 216 uniques categories.


In [45]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
toronto_onehot.shape

(2210, 216)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [47]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,Adelaide,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,...,0.00,0.00,0.000000,0.00000,0.00,0.020000,0.000000,0.0000,0.00,0.00
1,Bathurst Quay,0.000000,0.058824,0.058824,0.058824,0.058824,0.176471,0.117647,0.000000,0.000000,...,0.00,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.0000,0.00,0.00
2,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.00000,0.00,0.020000,0.000000,0.0000,0.00,0.00
3,Brockton,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.0000,0.00,0.00
4,Business Reply Mail Processing Centre 969 Eastern,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.0000,0.00,0.00
5,CN Tower,0.000000,0.058824,0.058824,0.058824,0.058824,0.176471,0.117647,0.000000,0.000000,...,0.00,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.0000,0.00,0.00
6,Cabbagetown,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022222,0.000000,...,0.00,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.0000,0.00,0.00
7,Central Bay Street,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,...,0.00,0.00,0.000000,0.00000,0.00,0.020000,0.000000,0.0000,0.00,0.00
8,Chinatown,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.02,0.000000,0.00000,0.00,0.040000,0.000000,0.0600,0.02,0.00
9,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.0000,0.00,0.00


In [48]:
toronto_grouped.shape

(72, 216)

#### Let's print each neighborhood along with the top 5 most common venues

In [49]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
              venue  freq
0  Asian Restaurant  0.06
1              Café  0.06
2       Coffee Shop  0.06
3        Steakhouse  0.06
4       Pizza Place  0.04


----Bathurst Quay----
              venue  freq
0   Airport Service  0.18
1  Airport Terminal  0.12
2          Boutique  0.06
3   Harbor / Marina  0.06
4     Boat or Ferry  0.06


----Berczy Park----
          venue  freq
0   Coffee Shop  0.10
1      Beer Bar  0.04
2        Bakery  0.04
3  Cocktail Bar  0.04
4   Cheese Shop  0.04


----Brockton----
            venue  freq
0  Breakfast Spot  0.09
1            Café  0.09
2     Coffee Shop  0.09
3      Restaurant  0.05
4   Burrito Place  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.12
1         Yoga Studio  0.06
2          Restaurant  0.06
3             Brewery  0.06
4          Skate Park  0.06


----CN Tower----
              venue  freq
0   Airport Service  0.18
1  Airport Terminal  0.12
2    

#### Define function get venues in order of preferences or votes

In [50]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Top 10 Venues per neighborhood

In [51]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Asian Restaurant,Coffee Shop,Café,Steakhouse,Gastropub,Bar,Hotel,Sushi Restaurant,Pizza Place,Japanese Restaurant
1,Bathurst Quay,Airport Service,Airport Terminal,Boat or Ferry,Boutique,Rental Car Location,Plane,Coffee Shop,Harbor / Marina,Bar,Sculpture Garden
2,Berczy Park,Coffee Shop,Bakery,Cheese Shop,Steakhouse,Café,Beer Bar,Seafood Restaurant,Farmers Market,Cocktail Bar,Italian Restaurant
3,Brockton,Coffee Shop,Breakfast Spot,Café,Bakery,Stadium,Sandwich Place,Burrito Place,Restaurant,Climbing Gym,Pet Store
4,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Auto Workshop,Park,Pizza Place,Comic Shop,Recording Studio,Restaurant,Butcher,Burrito Place,Brewery


#### K mean clustering for neighborhoods

In [77]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50] 

array([0, 4, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0,
       2, 0, 0, 0, 0, 0, 4, 0, 0, 4, 0, 0, 4, 2, 0, 3, 0, 0, 0, 0, 4, 0,
       0, 0, 0, 2, 1, 0], dtype=int32)

In [78]:
toronto_data.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M9A,Downtown Toronto,Queen's Park,43.667856,-79.532242
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418


#### Merge k means cluster set up with toronto data / neighborhood

In [80]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0.0,Coffee Shop,Park,Bakery,Pub,Restaurant,Mexican Restaurant,Café,Theater,Breakfast Spot,Brewery
1,M9A,Downtown Toronto,Queen's Park,43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937,0.0,Coffee Shop,Fast Food Restaurant,Café,Theater,Bookstore,Tea Room,Clothing Store,Ramen Restaurant,Music Venue,Beer Bar
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937,0.0,Coffee Shop,Fast Food Restaurant,Café,Theater,Bookstore,Tea Room,Clothing Store,Ramen Restaurant,Music Venue,Beer Bar
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0.0,Coffee Shop,Café,Restaurant,Italian Restaurant,Park,Japanese Restaurant,Bakery,Pizza Place,Gastropub,Farmers Market


In [82]:
toronto_merged = toronto_merged.dropna(subset=['Cluster Labels']) #drop NaN from cluster label columns
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int) #convert float into Int
toronto_merged.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Pub,Restaurant,Mexican Restaurant,Café,Theater,Breakfast Spot,Brewery
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937,0,Coffee Shop,Fast Food Restaurant,Café,Theater,Bookstore,Tea Room,Clothing Store,Ramen Restaurant,Music Venue,Beer Bar
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937,0,Coffee Shop,Fast Food Restaurant,Café,Theater,Bookstore,Tea Room,Clothing Store,Ramen Restaurant,Music Venue,Beer Bar
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Restaurant,Italian Restaurant,Park,Japanese Restaurant,Bakery,Pizza Place,Gastropub,Farmers Market
5,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Health Food Store,Pub,Wings Joint,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run


#### Plot cluster data in diff colors

In [83]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Getting details from each clusters...

In [85]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Park,Bakery,Pub,Restaurant,Mexican Restaurant,Café,Theater,Breakfast Spot,Brewery
2,Downtown Toronto,0,Coffee Shop,Fast Food Restaurant,Café,Theater,Bookstore,Tea Room,Clothing Store,Ramen Restaurant,Music Venue,Beer Bar
3,Downtown Toronto,0,Coffee Shop,Fast Food Restaurant,Café,Theater,Bookstore,Tea Room,Clothing Store,Ramen Restaurant,Music Venue,Beer Bar
4,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Italian Restaurant,Park,Japanese Restaurant,Bakery,Pizza Place,Gastropub,Farmers Market
5,East Toronto,0,Trail,Health Food Store,Pub,Wings Joint,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run
6,Downtown Toronto,0,Coffee Shop,Bakery,Cheese Shop,Steakhouse,Café,Beer Bar,Seafood Restaurant,Farmers Market,Cocktail Bar,Italian Restaurant
7,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Chinese Restaurant,Ice Cream Shop,Bubble Tea Shop,Burger Joint,Café,Spa,Seafood Restaurant,Sandwich Place
8,Downtown Toronto,0,Café,Grocery Store,Coffee Shop,Park,Diner,Baby Store,Convenience Store,Italian Restaurant,Restaurant,Nightclub
9,Downtown Toronto,0,Asian Restaurant,Coffee Shop,Café,Steakhouse,Gastropub,Bar,Hotel,Sushi Restaurant,Pizza Place,Japanese Restaurant
10,Downtown Toronto,0,Asian Restaurant,Coffee Shop,Café,Steakhouse,Gastropub,Bar,Hotel,Sushi Restaurant,Pizza Place,Japanese Restaurant


In [86]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Central Toronto,1,Garden,Wings Joint,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run,Discount Store


In [87]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,Central Toronto,2,Bus Line,Park,Dim Sum Restaurant,Swim School,Wings Joint,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run
34,Central Toronto,2,Bus Line,Park,Trail,Jewelry Store,Sushi Restaurant,Wings Joint,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run
35,Central Toronto,2,Bus Line,Park,Trail,Jewelry Store,Sushi Restaurant,Wings Joint,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run
66,Downtown Toronto,2,Park,Playground,Trail,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run,Discount Store


In [88]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
49,Central Toronto,3,Playground,Intersection,Restaurant,Tennis Court,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run,Discount Store
50,Central Toronto,3,Playground,Intersection,Restaurant,Tennis Court,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run,Discount Store


In [89]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
59,Downtown Toronto,4,Airport Service,Airport Terminal,Boat or Ferry,Boutique,Rental Car Location,Plane,Coffee Shop,Harbor / Marina,Bar,Sculpture Garden
60,Downtown Toronto,4,Airport Service,Airport Terminal,Boat or Ferry,Boutique,Rental Car Location,Plane,Coffee Shop,Harbor / Marina,Bar,Sculpture Garden
61,Downtown Toronto,4,Airport Service,Airport Terminal,Boat or Ferry,Boutique,Rental Car Location,Plane,Coffee Shop,Harbor / Marina,Bar,Sculpture Garden
62,Downtown Toronto,4,Airport Service,Airport Terminal,Boat or Ferry,Boutique,Rental Car Location,Plane,Coffee Shop,Harbor / Marina,Bar,Sculpture Garden
63,Downtown Toronto,4,Airport Service,Airport Terminal,Boat or Ferry,Boutique,Rental Car Location,Plane,Coffee Shop,Harbor / Marina,Bar,Sculpture Garden
64,Downtown Toronto,4,Airport Service,Airport Terminal,Boat or Ferry,Boutique,Rental Car Location,Plane,Coffee Shop,Harbor / Marina,Bar,Sculpture Garden
65,Downtown Toronto,4,Airport Service,Airport Terminal,Boat or Ferry,Boutique,Rental Car Location,Plane,Coffee Shop,Harbor / Marina,Bar,Sculpture Garden
